In [1]:
# Core
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline

# Astronomy
from astropy.time import Time
from astropy.units import deg

# Utility
import os
from datetime import date
import time
from tqdm.auto import tqdm

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import Image

# Libraries for getting Alerce data out of ZTF2 database
import json
import psycopg2
from alerce.api import AlerceAPI

# MSE imports
from utils import range_inc
from astro_utils import date_to_mjd, mjd_to_date
from ztf_data import load_ztf_det, load_ztf_det_year, ztf_det_add_dir, load_ztf_det_all
# from ra_dec import radec2dir, radec_diff, direction_diff
# from horizons_files import load_ast_obs_jpl
# from asteroid_data import make_data_one_file, get_earth_pos
from asteroid_dataframe import spline_ast_vec_obs

In [2]:
# Set plot style variables
mpl.rcParams['figure.figsize'] = [16.0, 10.0]
mpl.rcParams['font.size'] = 16

### Load Detections from Alerce ZTF2 Database

In [3]:
ztf, mjd_unq = load_ztf_det_all()

Loaded ../data/ztf/ztf-detections.h5 from disk.


In [4]:
# Review DataFrame
ztf

,ObjectID,CandidateID,TimeStampID,mjd,ra,dec,ux,uy,uz,mag_app,asteroid_prob
0,b'ZTF18acebhfp',676397301515010013,14490,58430.397303,41.357345,58.879488,0.387942,0.653853,0.649598,18.946699,0.865682
1,b'ZTF18abodmwk',596403415715010014,5831,58350.403414,30.969721,65.305308,0.358224,0.558644,0.748059,19.010401,0.855504
2,b'ZTF18abodmwk',626428345715010011,10614,58380.428345,30.969705,65.305294,0.358224,0.558644,0.748059,18.935900,0.855504
3,b'ZTF18abodmwk',630507595715015045,11250,58384.507593,30.969940,65.305305,0.358223,0.558645,0.748059,19.260401,0.855504
4,b'ZTF18abodmwk',618384965715010022,9040,58372.384965,30.969643,65.305179,0.358226,0.558644,0.748058,19.220200,0.855504
...,...,...,...,...,...,...,...,...,...,...,...
5459014,b'ZTF20aareruw',1151532523515015015,97109,58905.532523,253.007910,55.485537,-0.165587,-0.169403,0.971537,19.192400,0.608023
5459015,b'ZTF20aarerwx',1151533002615015009,97110,58905.533009,232.886408,53.509617,-0.358833,-0.115301,0.926253,19.687099,0.559474
5459016,b'ZTF20aarerww',1151533002115010003,97110,58905.533009,236.167899,54.618457,-0.322375,-0.116973,0.939357,19.957001,0.392662
5459017,b'ZTF20aarervr',1151526063515015015,97098,58905.526065,286.235286,33.876902,0.232120,-0.509626,0.828494,19.049299,0.517241


In [5]:
# Review data types
ztf.dtypes

ObjectID            |S12
CandidateID        int64
TimeStampID        int32
mjd              float64
ra               float64
dec              float64
ux               float64
uy               float64
uz               float64
mag_app          float64
asteroid_prob    float64
dtype: object

In [13]:
# Sanity check: we should be able to recover the mjd of an observation by indexing into mjd_unq
assert np.all(mjd_unq[ztf.TimeStampID] == ztf.mjd)

### Summarize Observations by Month

In [16]:
def obs_by_month(ztf):
    """Generate a chart summarizing observations by month in ZTF data"""
    # Extract the year-month tuple for each observation for summarizing
    tt = Time(ztf.mjd, format='mjd')
    isotimes = tt.iso
    ym = np.array([isotime[0:7] for isotime in isotimes])
    ym_ser = pd.Series(data=ym, index=ztf.index)

    # Group data by month for monthly summary
    obs_monthly = ztf.groupby(ym_ser)
    obs_monthly_count = obs_monthly.size()

    # Calculations for plot
    month_strs = obs_monthly_count.index.values
    x_values = np.arange(obs_monthly_count.size)
    x_dates = [date(int(x[0:4]), int(x[5:7]), 1) for x in month_strs]
    y_values = obs_monthly_count.values

    # Plot the number of observations by month
    fig, ax = plt.subplots()
    ax.set_title('Alerce Asteroid Observations by Month')
    ax.set_xlabel('Month')
    ax.set_ylabel('Asteroid Observations')
    # ax.bar(x=x_values, height=y_values, tick_label=month_strs, color='blue')
    ax.bar(x=x_values, height=y_values, color='blue')
    ax.set_xticks(x_values[::3])
    ax.set_xticks(x_values, minor=True)
    ax.set_xticklabels(month_strs[::3], minor=False)
    # ax.legend()
    ax.grid()
    fig.savefig('../figs/alerce/alerce_ast_per_month.png', bbox_inches='tight')
    plt.show()

In [18]:
# Display the bar chart
Image(filename='../figs/alerce/alerce_ast_per_month.png')

### Extract key Data from ZTF Frame

In [19]:
# Extract mjd, ra, and dec of the ZTF observations as arrays of astropy angles
# mjd_ztf = ztf.mjd.values
# ra_ztf = ztf.ra.values
# dec_ztf = ztf.dec.values

In [20]:
# Extract directions of the ZTF observations as an Nx3 array
u_ztf = ztf[['ux', 'uy', 'uz']].values

### Load MSE Calculated Asteroid Directions for Comparison to ZTF

In [22]:
# Date range in ZTF data
mjd_min = np.min(mjd_unq)
mjd_max = np.max(mjd_unq)
dt_min = mjd_to_date(mjd_min)
dt_max = mjd_to_date(mjd_max)
print(f'ZTF mjd range : {mjd_min:9.3f} to {mjd_max:9.3f}')
print(f'ZTF date range: {dt_min} to {dt_max}')

ZTF mjd range : 58270.170 to 58905.533
ZTF date range: 2018-06-01 to 2020-02-26


In [23]:
# Range of asteroids to compare to ZTF
n0: int = 1
n1: int = 16
    
# Observatory site
site_name = 'palomar'

In [24]:
# Build splined positions and observations against unique observation times
ast_pos, earth_pos, ast_dir = spline_ast_vec_obs(n0=n0, n1=n1, mjd=mjd_unq, site_name=site_name)

In [25]:
mjd_unq.size

97111

In [ ]:
# Alias inputs to interp_ast_dir
ast_num_src = jpl.asteroid_num.values
mjd_src = jpl.mjd.values
u_src = jpl[['ux_jpl', 'uy_jpl', 'uz_jpl']].values
ast_num_out = 1
mjd_out = mjd_ztf

In [ ]:
# Splined direction of this asteroid according to JPL
u_jpl = interp_ast_dir(ast_num_src, mjd_src, u_src, ast_num_out, mjd_out)

In [ ]:
u_jpl.shape

In [ ]:
ztf

In [ ]:
def compare_ztf_src(ztf: pd.DataFrame, ast_num_src: np.ndarray, mjd_src: np.ndarray, u_src: np.ndarray):
    """
    Construct splined predicted asteroid directions from a source at desired dates.
    INPUTS:
        ztf        : DataFrame of ZTF observations; columns must include mjd, ux, uy, uz
        ast_num_src: asteroid numbers whose position is predicted by source; shape (N,)
        mjd_src    : modified julian dates as of which direction is predicted by source; shape (N,)
        u_src      : directions from observatory to asteroid predicted by source; shape (N,3,)
    OUTPUTS:
        ast_nums   : array of distinct asteroids whose distance is compared; shape (K,)
        angle_diff : difference in angle between each ZTF observation and splined asteroid position; shape (N,K,)
    """
    pass


In [ ]:
# Get distinct asteroid numbers in source
ast_nums = np.unique(ast_num_src)
K = ast_nums.size

# Number of rows in ZTF data
M = ztf.shape[0]

# Initialize empty array of distances
angle_diff = np.zeros(shape=(M,K))

# Array of times to be interpolated from ztf
mjd_out = ztf.mjd.values

# Extract directions of the ZTF observations as an Mx3 array
u_ztf = ztf[['ux', 'uy', 'uz']].values

In [ ]:
ast_nums

In [ ]:
K

In [ ]:
M

In [ ]:
# Iterate over asteroids one at a time
for k, ast_num in enumerate(ast_nums):
    # The interpolated direction of this asteroid at the ZTF observation times
    u_out = interp_ast_dir(ast_num_src=ast_num_src, mjd_src=mjd_src, u_src=u_src, 
                           ast_num_out=ast_num, mjd_out=mjd_out)
    # Distance from u_ztf to u_out
    u_dist = np.linalg.norm(u_out - u_ztf, axis=1)
    
    # Convert to arc seconds and save to column k
    # angle_diff[:, k] = 2.0*np.arcsin(u_dist*0.5)
    angle_diff[:, k] = u_dist

In [ ]:
angle_diff.shape

In [ ]:
np.min(angle_diff)

In [ ]:
np.max(angle_diff)

In [ ]:
u_dist.shape

In [ ]:
np.max(u_out)

In [ ]:
u_out.shape

In [ ]:
u_out[0]

In [ ]:
np.linalg.norm(u_out, axis=1)

In [ ]:
u_out_sz = np.linalg.norm(u_out, axis=1)

In [ ]:
np.argmax(u_out_sz)

In [ ]:
i = np.argmax(u_out_sz)

In [ ]:
u_out[i]

In [ ]:
mjd[i]

In [ ]:
mjd_out[i]

In [ ]:
ast_num_out = ast_num
mask = (ast_num_src == ast_num_out)

In [ ]:
x_spline = mjd_src[mask]
y_spline = u_src[mask]
u_spline = CubicSpline(x=x_spline, y=y_spline)

In [ ]:
np.min(x_spline)

In [ ]:
np.max(x_spline)